In [294]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from utils.helpers import md

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import requests

BACKEND_URL = "http://127.0.0.1:5001/"
youtube_url = "https://www.youtube.com/watch?v=QlUcyaXiXPE&ab_channel=MorningBrewDaily"

response = requests.post(BACKEND_URL, json={"youtube_url": youtube_url})
result = response.json()
result

In [180]:
from data.get_youtube_data import YouTubeDataFetcher

youtube_url = "https://www.youtube.com/watch?v=QlUcyaXiXPE&ab_channel=MorningBrewDaily"
youtube = YouTubeDataFetcher()
meta_data = youtube.get_meta_data(youtube_url)

2024-12-25 14:49:33,452 - INFO - file_cache is only supported with oauth2client<4.0.0
2024-12-25 14:49:34,437 - INFO - Language: English
2024-12-25 14:49:34,438 - INFO - Language: en
2024-12-25 14:49:34,439 - INFO - Generated automatically: False
2024-12-25 14:49:34,440 - INFO - Translation available: True
2024-12-25 14:49:34,441 - INFO - -----
2024-12-25 14:49:34,442 - INFO - Found manual English transcript


In [310]:
meta_data

MetaData(title='Tech CEOs Charm Trump With Big Bucks', channel_name='Morning Brew Daily', publish_date='2024-12-16', chapters=['00:00 - Patterson bookstore bonus extravaganza', '2:15 - Tech CEOs donate to Trump', '6:40 - Broadcom quietly cashin’ in', '10:45 - What is up with drones??', '15:10 - Trump wants to end Daylight Saving Time', '18:30 - Hot Wheels lead the way', '21:30 - Week Ahead'], transcript={'transcript': [{'text': 'Good Morning Brew Daily Show.', 'start': 0.066, 'duration': 2.303}, {'text': "I'm Neal Freyman. And I'm Toby Howell.", 'start': 2.369, 'duration': 1.735}, {'text': "today. It's not Barbie.", 'start': 4.104, 'duration': 1.434}, {'text': "It's not Lego.", 'start': 5.538, 'duration': 1.035}, {'text': 'The best selling toy in the world\nis. Yeah.', 'start': 6.573, 'duration': 3.003}, {'text': "You're going to have to wait for that\nthen. It's a bird.", 'start': 9.576, 'duration': 2.202}, {'text': "It's a plane.", 'start': 11.778, 'duration': 0.634}, {'text': "No, i

In [208]:
from data.preprocess import clean_text, clean_transcript
meta_data.transcript['transcript'][-6]
clean_text(meta_data.transcript['transcript'][-6]['text'])

{'text': "Hair makeup's got a dinner down\nin Mar-A-Lago.",
 'start': 1502.601,
 'duration': 2.836}

"Hair makeup's got a dinner down in Mar-A-Lago."

In [249]:
from data.preprocess import merge_chapter_transcript

chapter_transcript = merge_chapter_transcript(meta_data)
chapter_transcript[-1].transcript[-6:]

["Hair makeup's got a dinner down in Mar-A-Lago.",
 'Really?',
 "He's inviting anyone.",
 'Devin Emery is our chief content officer.',
 'And our show is a production of Morning Brew.',
 "Great show. Danielle. Let's run it back tomorrow."]

## LangChain

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import langchain_community
langchain_community.__version__

'0.3.13'

In [250]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the document into chunks with a specified chunk size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = []
for chapter in chapter_transcript:
    all_splits.extend(text_splitter.split_text(' '.join(chapter.transcript)))

all_splits[:10]
# Store the document into a vector store with a specific embedding model
vectorstore = FAISS.from_texts(all_splits, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

["Good Morning Brew Daily Show. I'm Neal Freyman. And I'm Toby Howell. today. It's not Barbie. It's not Lego. The best selling toy in the world is. Yeah. You're going to have to wait for that then. It's a bird. It's a plane. No, it's a drone hovering over new Jersey. But in reality, it's probably a plane. It's Monday, December 16th. Let's ride. The holiday season is full of heartwarming stories of generosity and giving back. And I got another one for you right here last week. The extremely popular",
 "you right here last week. The extremely popular author James Patterson handed out $500 holiday bonuses to 600 employees at independent bookstores across the country. It's a tradition he started in 2015, inviting thousands of booksellers to apply and picking hundreds of winners from the nominees. In a statement, Patterson said booksellers save lives, period. I'm happy to be able to acknowledge them and all their hard work this holiday season. I mean, it does warm the heart. James Patterson

2024-12-25 15:09:23,851 - INFO - Use pytorch device_name: mps
2024-12-25 15:09:23,852 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


In [251]:
question = "who are the speakers of this podcast"
results = vectorstore.similarity_search_with_score(
    question,
    k=10,
)
results

[(Document(id='78f93b32-2d36-4b25-afa6-d085aff833c5', metadata={}, page_content="thinking about these things, so share this pod. Give them the drone conversation time stamp and just say thoughts. I think the rest will take care of itself. Let's roll the credits. Emily Milian is our executive producer. Raven live is our producer. Olivia Graham is our associate producer. You Chenoa is our technical director. Billy Menino is on audio. Hair makeup's got a dinner down in Mar-A-Lago. Really? He's inviting anyone. Devin Emery is our chief content officer. And our show is a"),
  0.85989475),
 (Document(id='30c4e6cd-3635-44f5-b726-7d0f4e8b067b', metadata={}, page_content="questions, comments or feedback, send an email to Morning Brew Daily at Morning brew.com. And if you're a satisfied customer and want to help grow this podcast, share it with friends, family, and coworkers who are lacking a solid news roundup in the morning. For everyone with shares block, Toby's got our recommendation. I want

In [271]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10})
retriever.invoke(question)

[Document(id='78f93b32-2d36-4b25-afa6-d085aff833c5', metadata={}, page_content="thinking about these things, so share this pod. Give them the drone conversation time stamp and just say thoughts. I think the rest will take care of itself. Let's roll the credits. Emily Milian is our executive producer. Raven live is our producer. Olivia Graham is our associate producer. You Chenoa is our technical director. Billy Menino is on audio. Hair makeup's got a dinner down in Mar-A-Lago. Really? He's inviting anyone. Devin Emery is our chief content officer. And our show is a"),
 Document(id='07059542-ac4d-4671-9f32-f8aa01bed28d', metadata={}, page_content='pick one way or the other, right? And I think the way, based off his previous statements is he wants to make DST permanent. My win over the weekend is Hot Wheels because they are on fire right now.'),
 Document(id='50b4d092-1b48-4013-ae9d-cb0898b8d3ea', metadata={}, page_content="you right here last week. The extremely popular author James Pat

In [270]:
# use ConversationalRetrievalChain to pass chat history for follow up questions
from langchain.chains import ConversationalRetrievalChain
from langchain_community.llms import Replicate
llm = Replicate(
    model="meta/meta-llama-3-8b-instruct",
    model_kwargs={"temperature": 0.0, "top_p": 1, "max_new_tokens":500}
)

chat_chain = ConversationalRetrievalChain.from_llm(llm, retriever, return_source_documents=True)

In [266]:
result = chat_chain({"question": question, "chat_history": []})
print(result['answer'])

2024-12-25 15:11:34,961 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:11:35,720 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/khjvvbgqxsrj00ckzjmaz3s6vc "HTTP/1.1 200 OK"




Based on the provided context, the speakers of this podcast are:

1. Neal Freyman
2. Toby Howell

They are the hosts of the Morning Brew Daily Show.


In [267]:
chat_history = [(question, result["answer"])]
followup = "what company do they work for?"
followup_answer = chat_chain({"question": followup, "chat_history": chat_history})
print(followup_answer['answer'])

2024-12-25 15:11:45,119 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:11:46,285 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/mdc6fqhzk9rj20ckzjm8sj65zw "HTTP/1.1 200 OK"
2024-12-25 15:11:48,246 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:11:49,012 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/1abnnd2bq1rj60ckzjma7v17tc "HTTP/1.1 200 OK"




According to the context, Neal Freyman and Toby Howell work for Morning Brew.


In [268]:
followup = 'who are the rest of the members on the team'
followup_answer = chat_chain({"question": followup, "chat_history": chat_history})
print(followup_answer['answer'])

2024-12-25 15:12:26,860 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:12:27,611 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/dtrcjn72nhrj40ckzjma23xa84 "HTTP/1.1 200 OK"
2024-12-25 15:12:29,432 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:12:30,761 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/fm34g57cphrj20ckzjma87760r "HTTP/1.1 200 OK"




According to the transcript, the other members of the Morning Brew Daily Show team are:

* Emily Milian, Executive Producer
* Raven Live, Producer
* Olivia Graham, Associate Producer
* You Chenoa, Technical Director
* Billy Menino, Audio
* Devin Emery, Chief Content Officer


In [269]:
followup = 'where is hair and makeup'
followup_answer = chat_chain({"question": followup, "chat_history": chat_history})
print(followup_answer['answer'])

2024-12-25 15:12:45,424 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:12:46,674 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/kyzs8tsb5hrj40ckzjmtnfdj90 "HTTP/1.1 200 OK"
2024-12-25 15:12:51,769 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:12:53,151 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/v7dhb6j3ynrj20ckzjmta4ahzm "HTTP/1.1 200 OK"




According to the context, the hair and makeup team is located in Mar-A-Lago.


In [288]:
chat_history = []

def chat(query):
    global chat_history
    response = chat_chain(
        {"question": query, "chat_history": chat_history}
    )
    # Update chat history
    chat_history.append((query, response["answer"]))
    return response['answer']



In [287]:
answer

{'question': 'Where is hair and makeup?',
 'chat_history': [('Who are the speakers of this podcast',
   '\n\nBased on the provided context, the speakers of this podcast are:\n\n* Neal Freyman\n* Toby Howell'),
  ('What company do they work for?',
   '\n\nAccording to the context, Neal Freyman and Toby Howell work for Morning Brew.'),
  ('Who are the rest of the members on the production team?',
   '\n\nAccording to the text, the other members of the production team of the podcast are:\n\n* Emily Milian, executive producer\n* Raven live, producer\n* Olivia Graham, associate producer\n* You Chenoa, technical director\n* Billy Menino, audio\n* Devin Emery, chief content officer'),
  ('Where is hair and makeup?',
   '\n\nAccording to the context, the hair and makeup team is "down in Mar-A-Lago".')],
 'answer': '\n\nAccording to the context, the hair and makeup team is "down in Mar-A-Lago".',
 'source_documents': [Document(id='78f93b32-2d36-4b25-afa6-d085aff833c5', metadata={}, page_content

In [297]:
chat_history = []

question = 'Who are the speakers of this podcast'
answer = chat(question)
print(question)
print(answer)

followup = 'What company do they work for?'
answer = chat(followup)
print(question)
print(answer)

followup = 'Who are the rest of the members on the production team?'
answer = chat(followup)
print(question)
print(answer)

followup = 'Where is hair and makeup?'
answer = chat(followup)
print(question)
print(answer)

2024-12-25 15:32:26,110 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:32:27,286 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/7v6scc1f55rj40ckzjxt4avx2r "HTTP/1.1 200 OK"
2024-12-25 15:32:28,133 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/7v6scc1f55rj40ckzjxt4avx2r "HTTP/1.1 200 OK"
2024-12-25 15:32:28,899 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/7v6scc1f55rj40ckzjxt4avx2r "HTTP/1.1 200 OK"


Who are the speakers of this podcast


Based on the provided context, the speakers of this podcast are:

* Neal Freyman
* Toby Howell


2024-12-25 15:32:29,608 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:32:30,386 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/vent7yswt1rj40ckzjxsfthm84 "HTTP/1.1 200 OK"
2024-12-25 15:32:31,144 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/vent7yswt1rj40ckzjxsfthm84 "HTTP/1.1 200 OK"
2024-12-25 15:32:31,922 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/vent7yswt1rj40ckzjxsfthm84 "HTTP/1.1 200 OK"
2024-12-25 15:32:33,451 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:32:34,208 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/mdesjv2bq1rj40ckzjxt10zc8c "HTTP/1.1 200 OK"
2024-12-25 15:32:34,970 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/mdesjv2bq1rj40ckzjxt10zc8c "HTTP/1.1 200 OK"
2024-12-2

Who are the speakers of this podcast


According to the context, Neal Freyman and Toby Howell work for Morning Brew.


2024-12-25 15:32:36,633 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:32:37,797 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/2fvg942r69rj40ckzjxvm12hcr "HTTP/1.1 200 OK"
2024-12-25 15:32:38,556 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/2fvg942r69rj40ckzjxvm12hcr "HTTP/1.1 200 OK"
2024-12-25 15:32:39,001 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:32:39,844 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/pkf4f9b1g9rj20ckzjxswvymac "HTTP/1.1 200 OK"
2024-12-25 15:32:40,600 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/pkf4f9b1g9rj20ckzjxswvymac "HTTP/1.1 200 OK"
2024-12-25 15:32:41,358 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/pkf4f9b1g9rj20ckzjxswvymac "HTTP/1.1 200 OK"
2024-12-2

Who are the speakers of this podcast


According to the text, the other members of the production team of the podcast are:

* Emily Milian, executive producer
* Raven live, producer
* Olivia Graham, associate producer
* You Chenoa, technical director
* Billy Menino, audio
* Devin Emery, chief content officer


2024-12-25 15:32:45,438 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:32:46,258 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/15rmyy3tbhrj00ckzjxt1aq8p4 "HTTP/1.1 200 OK"
2024-12-25 15:32:47,769 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:32:49,024 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/ckg57jc3qdrj00ckzjxvw733nr "HTTP/1.1 200 OK"
2024-12-25 15:32:49,782 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/ckg57jc3qdrj00ckzjxvw733nr "HTTP/1.1 200 OK"


Who are the speakers of this podcast


According to the context, the hair and makeup team is "down in Mar-A-Lago".


### test functions

In [299]:
from data.get_youtube_data import YouTubeDataFetcher
from data.preprocess import merge_chapter_transcript

youtube_url = "https://www.youtube.com/watch?v=QlUcyaXiXPE&ab_channel=MorningBrewDaily"
youtube = YouTubeDataFetcher()
meta_data = youtube.get_meta_data(youtube_url)

chapter_transcript = merge_chapter_transcript(meta_data)
chapter_transcript

2024-12-25 16:07:31,187 - INFO - file_cache is only supported with oauth2client<4.0.0
2024-12-25 16:07:32,180 - INFO - Language: English
2024-12-25 16:07:32,181 - INFO - Language: en
2024-12-25 16:07:32,182 - INFO - Generated automatically: False
2024-12-25 16:07:32,183 - INFO - Translation available: True
2024-12-25 16:07:32,184 - INFO - -----
2024-12-25 16:07:32,185 - INFO - Found manual English transcript


[ChapterTranscript(chapter='Patterson bookstore bonus extravaganza', timestamp='00:00', transcript=['Good Morning Brew Daily Show.', "I'm Neal Freyman. And I'm Toby Howell.", "today. It's not Barbie.", "It's not Lego.", 'The best selling toy in the world is. Yeah.', "You're going to have to wait for that then. It's a bird.", "It's a plane.", "No, it's a drone hovering over new Jersey.", "But in reality, it's probably a plane.", "It's Monday, December 16th.", "Let's ride.", 'The holiday season is', 'full of heartwarming stories of generosity and giving back.', 'And I got another one for you right here last week.', 'The extremely popular author James Patterson handed out $500 holiday', 'bonuses to 600 employees at independent bookstores across the country.', "It's a tradition he started in 2015, inviting thousands of booksellers", 'to apply and picking hundreds of winners from the nominees.', 'In a statement, Patterson said booksellers save lives, period.', "I'm happy to be able to ackno

In [307]:
response

{'question': 'Where is hair and makeup?',
 'chat_history': [('Who are the speakers of this podcast',
   '\n\nBased on the provided context, the speakers of this podcast are:\n\n* Neal Freyman\n* Toby Howell'),
  ('What company do they work for?',
   '\n\nAccording to the context, Neal Freyman and Toby Howell work for Morning Brew.'),
  ('Who are the rest of the members on the production team?',
   '\n\nAccording to the text, the other members of the production team of the podcast are:\n\n* Emily Milian, executive producer\n* Raven live, producer\n* Olivia Graham, associate producer\n* You Chenoa, technical director\n* Billy Menino, audio\n* Devin Emery, chief content officer'),
  ('Where is hair and makeup?',
   '\n\nAccording to the context, the hair and makeup team is "down in Mar-A-Lago".')],
 'answer': '\n\nAccording to the context, the hair and makeup team is "down in Mar-A-Lago".',
 'source_documents': [Document(id='a3929022-0a9a-4dc5-b71f-8ac30acb7c7f', metadata={}, page_content

In [306]:
from src.vectorstore import get_retriever
from src.chat import chat, get_chat_chain

retriever = get_retriever(chapter_transcript)
chat_chain = get_chat_chain(retriever)

chat_history = []
query = 'Who are the speakers of this podcast'
response, chat_history = chat(query, chat_chain, chat_history)
print(query)
print(response['answer'])

query = 'What company do they work for?'
response, chat_history = chat(query, chat_chain, chat_history)
print(query)
print(response['answer'])

query = 'Who are the rest of the members on the production team?'
response, chat_history = chat(query, chat_chain, chat_history)
print(query)
print(response['answer'])

query = 'Where is hair and makeup?'
response, chat_history = chat(query, chat_chain, chat_history)
print(query)
print(response['answer'])


2024-12-25 16:13:42,322 - INFO - Use pytorch device_name: mps
2024-12-25 16:13:42,325 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2024-12-25 16:13:57,594 - INFO - HTTP Request: GET https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct "HTTP/1.1 200 OK"
2024-12-25 16:13:58,523 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 16:13:59,947 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/g7k62xhq1drj40ckzkgv8acfy4 "HTTP/1.1 200 OK"


Who are the speakers of this podcast


Based on the provided context, the speakers of this podcast are:

* Neal Freyman
* Toby Howell


2024-12-25 16:14:00,737 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 16:14:01,494 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/mbsvdf1zp1rj00ckzkgsedcx7m "HTTP/1.1 200 OK"
2024-12-25 16:14:01,914 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 16:14:02,672 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/mbwqfvj4jhrj00ckzkgv04fswr "HTTP/1.1 200 OK"


What company do they work for?


According to the context, Neal Freyman and Toby Howell work for Morning Brew.


2024-12-25 16:14:02,958 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 16:14:03,713 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/j5c4e928n9rj00ckzkgrcmavg0 "HTTP/1.1 200 OK"
2024-12-25 16:14:05,184 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 16:14:06,517 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/gtngcqthb1rj20ckzkgskvm484 "HTTP/1.1 200 OK"


Who are the rest of the members on the production team?


According to the text, the other members of the production team of the podcast are:

* Emily Milian, executive producer
* Raven live, producer
* Olivia Graham, associate producer
* You Chenoa, technical director
* Billy Menino, audio
* Devin Emery, chief content officer


2024-12-25 16:14:06,805 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 16:14:07,555 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/jz7ezgjqp1rj60ckzkgrzwgks4 "HTTP/1.1 200 OK"
2024-12-25 16:14:08,444 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 16:14:09,320 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/f884tejy2srj00ckzkgvs4p5z4 "HTTP/1.1 200 OK"


Where is hair and makeup?


According to the context, the hair and makeup team is "down in Mar-A-Lago".


## without LangChain

In [278]:
def generate_response_with_context(user_query):
    global chat_history
    # Retrieve relevant chunks
    retrieved_chunks = retrieve_relevant_chunks(user_query)
    
    # Augment prompt with history
    prompt = augment_prompt_with_history(user_query, retrieved_chunks)
    
    # Generate response
    bot_response = llm.invoke(prompt)
    
    chat_history.append({'user': question, 'bot': response})    
    return bot_response

def retrieve_relevant_chunks(user_query):
    retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 10})
    return [doc.page_content for doc in retriever.invoke(user_query)]

def augment_prompt_with_history(user_query, retrieved_chunks):
    global chat_history
    chat_history_text = format_history(chat_history)
    context = "\n\n".join(retrieved_chunks)
    return (
        f"Conversation History:\n{chat_history_text}\n\n"
        f"Context:\n{context}\n\n"
        f"User Query: {user_query}\n\n"
        f"Answer:"
    )

def format_history(chat_history):
    return "\n".join([f"User: {entry['user']}\nBot: {entry['bot']}" for entry in chat_history])

In [284]:
chat_history = []

question = 'Who are the speakers of this podcast'
answer = generate_response_with_context(question)
question
print(answer)

followup = 'What company do they work for?'
answer = generate_response_with_context(followup)
followup
print(answer)

followup = 'Who are the rest of the members on the production team?'
answer = generate_response_with_context(followup)
followup
print(answer)

followup = 'Where is hair and makeup?'
answer = generate_response_with_context(followup)
followup
print(answer)

2024-12-25 15:22:21,843 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:22:22,623 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/rhpyzc7pt1rj00ckzjrtm5rh3c "HTTP/1.1 200 OK"


'Who are the speakers of this podcast'



The speakers of this podcast are:

1. Neal Freyman
2. Toby Howell


2024-12-25 15:22:23,212 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:22:24,376 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/hf187q7w5hrj40ckzjrtb6w390 "HTTP/1.1 200 OK"
2024-12-25 15:22:25,133 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/hf187q7w5hrj40ckzjrtb6w390 "HTTP/1.1 200 OK"
2024-12-25 15:22:25,894 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/hf187q7w5hrj40ckzjrtb6w390 "HTTP/1.1 200 OK"
2024-12-25 15:22:27,223 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/hf187q7w5hrj40ckzjrtb6w390 "HTTP/1.1 200 OK"


'What company do they work for?'

...

'Who are the rest of the members on the production team?'



Based on the conversation history, the rest of the members on the production team are:

1. Emily Milian (executive producer)
2. Raven live (producer)
3. Olivia Graham (associate producer)
4. You Chenoa (technical director)
5. Billy Menino (audio)
6. Devin Emery (chief content officer)

These individuals are mentioned as part of the production team, but not as speakers on the podcast.


2024-12-25 15:22:30,395 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:22:31,219 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/4rgae8rqvhrj20ckzjs9z03gbc "HTTP/1.1 200 OK"


'Where is hair and makeup?'



According to the conversation history, Hair and Makeup is mentioned as being "down in Mar-A-Lago".


In [273]:
question
chat_history = []

retrieved_chunks = retrieve_relevant_chunks(question)
augmented_prompt = augment_prompt_with_history(question, retrieved_chunks)
print(augmented_prompt)
response = llm.invoke(augmented_prompt)
print(response)
chat_history.append({'user': question, 'bot': response})

'who are the speakers of this podcast'

Conversation History:


Context:
thinking about these things, so share this pod. Give them the drone conversation time stamp and just say thoughts. I think the rest will take care of itself. Let's roll the credits. Emily Milian is our executive producer. Raven live is our producer. Olivia Graham is our associate producer. You Chenoa is our technical director. Billy Menino is on audio. Hair makeup's got a dinner down in Mar-A-Lago. Really? He's inviting anyone. Devin Emery is our chief content officer. And our show is a

pick one way or the other, right? And I think the way, based off his previous statements is he wants to make DST permanent. My win over the weekend is Hot Wheels because they are on fire right now.

you right here last week. The extremely popular author James Patterson handed out $500 holiday bonuses to 600 employees at independent bookstores across the country. It's a tradition he started in 2015, inviting thousands of booksellers to apply and picking hundreds of winne

2024-12-25 15:13:35,863 - INFO - HTTP Request: GET https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct "HTTP/1.1 200 OK"
2024-12-25 15:13:36,587 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:13:37,763 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/nrpyxkfk15rj40ckzjmtdb3m48 "HTTP/1.1 200 OK"
2024-12-25 15:13:38,527 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/nrpyxkfk15rj40ckzjmtdb3m48 "HTTP/1.1 200 OK"
2024-12-25 15:13:39,286 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/nrpyxkfk15rj40ckzjmtdb3m48 "HTTP/1.1 200 OK"




Based on the conversation history, the speakers of this podcast are:

1. Neal Freyman
2. Toby Howell
3. James Patterson (author)
4. Elon Musk (tech mogul)
5. Sam Altman (tech mogul)
6. Mr. Beast (YouTuber)
7. Jeff Bezos (tech mogul)

Additionally, there are some mentions of other individuals, such as:

1. Emily Milian (executive producer)
2. Raven live (producer)
3. Olivia Graham (associate producer)
4. You Chenoa (technical director)
5. Billy Menino (audio)
6. Devin Emery (chief content officer)

Please note that some of these individuals may not be speakers on the podcast, but rather mentioned as part of the production team or other roles.


In [274]:
print(format_history(chat_history))

User: who are the speakers of this podcast
Bot: 

Based on the conversation history, the speakers of this podcast are:

1. Neal Freyman
2. Toby Howell
3. James Patterson (author)
4. Elon Musk (tech mogul)
5. Sam Altman (tech mogul)
6. Mr. Beast (YouTuber)
7. Jeff Bezos (tech mogul)

Additionally, there are some mentions of other individuals, such as:

1. Emily Milian (executive producer)
2. Raven live (producer)
3. Olivia Graham (associate producer)
4. You Chenoa (technical director)
5. Billy Menino (audio)
6. Devin Emery (chief content officer)

Please note that some of these individuals may not be speakers on the podcast, but rather mentioned as part of the production team or other roles.


In [261]:
followup_question = 'what company do they work for?'
retrieved_chunks = retrieve_relevant_chunks(followup_question)
augmented_prompt = augment_prompt_with_history(followup_question, retrieved_chunks)
print(augmented_prompt)
response = llm.invoke(augmented_prompt)
print(response)
chat_history.append({'user': followup_question, 'bot': response})

Conversation History:
User: who are the speakers of this podcast
Bot: 

Based on the conversation history, the speakers of this podcast are:

1. Neal Freyman
2. Toby Howell
3. James Patterson (author)
4. Elon Musk (tech mogul)
5. Sam Altman (tech mogul)
6. Mr. Beast (YouTuber)
7. Jeff Bezos (tech mogul)

Additionally, there are some mentions of other individuals, such as:

1. Emily Milian (executive producer)
2. Raven live (producer)
3. Olivia Graham (associate producer)
4. You Chenoa (technical director)
5. Billy Menino (audio)
6. Devin Emery (chief content officer)

Please note that some of these individuals may not be speakers on the podcast, but rather mentioned as part of the production team or other roles.

Context:
Today, America's top tech moguls are marching one by one to Mar a Lago, a pilgrimage they hope will repair a rocky relationship with president elect Trump and curry favor with the upcoming administration. Last week, it was Google's Sundar Pichai and Apple's Tim Cook m

2024-12-25 15:10:30,403 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:10:31,956 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/t47c2f0vqhrj60ckzjkr7d95zg "HTTP/1.1 200 OK"
2024-12-25 15:10:32,715 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/t47c2f0vqhrj60ckzjkr7d95zg "HTTP/1.1 200 OK"
2024-12-25 15:10:33,476 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/t47c2f0vqhrj60ckzjkr7d95zg "HTTP/1.1 200 OK"
2024-12-25 15:10:34,244 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/t47c2f0vqhrj60ckzjkr7d95zg "HTTP/1.1 200 OK"
2024-12-25 15:10:35,005 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/t47c2f0vqhrj60ckzjkr7d95zg "HTTP/1.1 200 OK"




Based on the conversation history, the speakers of this podcast are:

1. Neal Freyman
2. Toby Howell
3. James Patterson (author)
4. Elon Musk (tech mogul)
5. Sam Altman (tech mogul)
6. Mr. Beast (YouTuber)
7. Jeff Bezos (tech mogul)

Additionally, there are some mentions of other individuals, such as:

1. Emily Milian (executive producer)
2. Raven live (producer)
3. Olivia Graham (associate producer)
4. You Chenoa (technical director)
5. Billy Menino (audio)
6. Devin Emery (chief content officer)

As for the company they work for, based on the conversation, it is mentioned that:

* Devin Emery is the chief content officer of Morning Brew.
* The podcast is a production of Morning Brew.

So, Neal Freyman, Toby Howell, and Devin Emery work for Morning Brew.


In [262]:
followup_question = 'who are the rest of the members on the team'
retrieved_chunks = retrieve_relevant_chunks(followup_question)
augmented_prompt = augment_prompt_with_history(followup_question, retrieved_chunks)
print(augmented_prompt)
response = llm.invoke(augmented_prompt)
print(response)
chat_history.append({'user': followup_question, 'bot': response})

Conversation History:
User: who are the speakers of this podcast
Bot: 

Based on the conversation history, the speakers of this podcast are:

1. Neal Freyman
2. Toby Howell
3. James Patterson (author)
4. Elon Musk (tech mogul)
5. Sam Altman (tech mogul)
6. Mr. Beast (YouTuber)
7. Jeff Bezos (tech mogul)

Additionally, there are some mentions of other individuals, such as:

1. Emily Milian (executive producer)
2. Raven live (producer)
3. Olivia Graham (associate producer)
4. You Chenoa (technical director)
5. Billy Menino (audio)
6. Devin Emery (chief content officer)

Please note that some of these individuals may not be speakers on the podcast, but rather mentioned as part of the production team or other roles.
User: what company do they work for?
Bot: 

Based on the conversation history, the speakers of this podcast are:

1. Neal Freyman
2. Toby Howell
3. James Patterson (author)
4. Elon Musk (tech mogul)
5. Sam Altman (tech mogul)
6. Mr. Beast (YouTuber)
7. Jeff Bezos (tech mogul)



2024-12-25 15:10:40,865 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:10:42,045 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/14snqbj4enrj60ckzjktz9wefg "HTTP/1.1 200 OK"
2024-12-25 15:10:42,807 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/14snqbj4enrj60ckzjktz9wefg "HTTP/1.1 200 OK"
2024-12-25 15:10:43,659 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/14snqbj4enrj60ckzjktz9wefg "HTTP/1.1 200 OK"
2024-12-25 15:10:44,458 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/14snqbj4enrj60ckzjktz9wefg "HTTP/1.1 200 OK"
2024-12-25 15:10:45,225 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/14snqbj4enrj60ckzjktz9wefg "HTTP/1.1 200 OK"
2024-12-25 15:10:46,085 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/14snqbj4enrj60ckzjktz9wefg "HTTP/1.1 200 OK"
2024-12-25 15:10:47,523 -



Based on the conversation history, the speakers of this podcast are:

1. Neal Freyman
2. Toby Howell
3. James Patterson (author)
4. Elon Musk (tech mogul)
5. Sam Altman (tech mogul)
6. Mr. Beast (YouTuber)
7. Jeff Bezos (tech mogul)

Additionally, there are some mentions of other individuals, such as:

1. Emily Milian (executive producer)
2. Raven live (producer)
3. Olivia Graham (associate producer)
4. You Chenoa (technical director)
5. Billy Menino (audio)
6. Devin Emery (chief content officer)

As for the company they work for, based on the conversation, it is mentioned that:

* Devin Emery is the chief content officer of Morning Brew.
* The podcast is a production of Morning Brew.

So, Neal Freyman, Toby Howell, and Devin Emery work for Morning Brew.

The rest of the members on the team are:

* Emily Milian (executive producer)
* Raven live (producer)
* Olivia Graham (associate producer)
* You Chenoa (technical director)
* Billy Menino (audio)

These individuals are part of the p

In [263]:
followup_question = "where is hair and makeup?"
retrieved_chunks = retrieve_relevant_chunks(followup_question)
augmented_prompt = augment_prompt_with_history(followup_question, retrieved_chunks)
print(augmented_prompt)
response = llm.invoke(augmented_prompt)
print(response)
chat_history.append({'user': followup_question, 'bot': response})

Conversation History:
User: who are the speakers of this podcast
Bot: 

Based on the conversation history, the speakers of this podcast are:

1. Neal Freyman
2. Toby Howell
3. James Patterson (author)
4. Elon Musk (tech mogul)
5. Sam Altman (tech mogul)
6. Mr. Beast (YouTuber)
7. Jeff Bezos (tech mogul)

Additionally, there are some mentions of other individuals, such as:

1. Emily Milian (executive producer)
2. Raven live (producer)
3. Olivia Graham (associate producer)
4. You Chenoa (technical director)
5. Billy Menino (audio)
6. Devin Emery (chief content officer)

Please note that some of these individuals may not be speakers on the podcast, but rather mentioned as part of the production team or other roles.
User: what company do they work for?
Bot: 

Based on the conversation history, the speakers of this podcast are:

1. Neal Freyman
2. Toby Howell
3. James Patterson (author)
4. Elon Musk (tech mogul)
5. Sam Altman (tech mogul)
6. Mr. Beast (YouTuber)
7. Jeff Bezos (tech mogul)



2024-12-25 15:10:53,517 - INFO - HTTP Request: POST https://api.replicate.com/v1/models/meta/meta-llama-3-8b-instruct/predictions "HTTP/1.1 201 Created"
2024-12-25 15:10:54,849 - INFO - HTTP Request: GET https://api.replicate.com/v1/predictions/cjgcysvnzxrj40ckzjkvqdw9jm "HTTP/1.1 200 OK"




According to the conversation history, Hair and Makeup is mentioned as being down in Mar-A-Lago.
